### 1. Cite 5 diferenças entre o AdaBoost e o GBM.

|AdaBoost| GBM|
|-------------|---------|
|Floresta de _"Stumps"_|Floresta de árvores |
|Primeiro passo é um _"Stumps"_ |Primeiro passo é a média|
|Respostas têm pesos diferentes|Todas respostas têm um multiplicador em comum chamado _learning rate_|
|Utiliza várias funções de perda tornando o algorítmo sensível aos outliers |Qualquer função de perda diferenciável pode ser utilizada, o que torna o algorítmo mais robusto para outliers|
|Cada classificador tem diferentes pesos atribuídos à previsão final com base em seu desempenho |Todos os classificadores são pesados igualmente e sua capacidade preditiva é restrita com taxa de aprendizado para aumentar a precisão |


### 2. Acesse o link Scikit-learn – GBM, leia a explicação (traduza se for preciso) e crie um jupyter notebook contendo o exemplo de classificação e de regressão do GBM.

In [1]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=1.0,
    max_depth=1, 
    random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

In [3]:
ini_array = y
  
unique, frequency = np.unique(ini_array,  
                              return_counts = True) 
print("Unique Values...:",  
      unique) 
print("Frequency Values:", 
      frequency)
print(f'Total de valores:{y.shape}')

Unique Values...: [-1.  1.]
Frequency Values: [6068 5932]
Total de valores:(12000,)


### 3. Cite 5 Hyperparametros importantes no GBM.

- **loss**: Função de perda a ser otimizada, podendo ser: **log_loss** quando o desvio for binomial e multidirecional ou **exponencial**;
- **learning_rate**: A taxa de aprendizado diminui a contribuição de cada árvore. Há uma compensação entre learning_rate e n_estimators, devendo os valores estar no intervalo de 0.0 a infinito;
- **min_samples_split**: O número mínimo de amostras necessárias para dividir um nó interno;
- **min_samples_leaf**: O número mínimo de amostras necessárias para estar em um nó folha;
- **max_depth**: Profundidade máxima dos estimadores de regressão individuais;
- **max_features**: O número de recursos a serem considerados ao procurar a melhor divisão;
- **criterion**: A função para medir a qualidade de uma divisão. Os critérios suportados são **friedman_mse** para o erro quadrático médio com pontuação de melhoria de Friedman, **squared_error** para o erro quadrático médio. O valor padrão de **friedman_mse** geralmente é o melhor, pois pode fornecer uma aproximação melhor em alguns casos;
- **subsample**: Fração de amostras a ser usadas para ajustar os _base learners_ individuais. Se este valor for menor que 1.0 o resultado será igual ao **Stochastic Gradient Boosting**. O **subsample** interage com o parâmetro **n_estimators**. Ao escolher valores menores que 1.0 haverá uma redução da variação e um aumento do viés. Os valores precisam estar entre 0.0 e 1.0;
- **n_estimators**: O número de estágios de **boosting** a serem executados. O reforço de gradiente é bastante robusto ao ajuste excessivo, portanto, um grande número geralmente resulta em melhor desempenho. Os valores devem estar no intervalo entre 1 e inf.



### 4. (Opcional) Utilize o GridSearch para encontrar os melhores hyperparametros para o conjunto de dados do exemplo

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer


In [5]:
%%time
#Parâmetros de Scoring: 
scoring = {'accuracy':  make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall':    make_scorer(recall_score)}

# Parâmetros para amostras
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],#list(np.arange (0.1, 3, 0.5)),
    "min_samples_split": np.arange(1, 55, 5),
    "min_samples_leaf": np.arange(1, 55, 5),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[5]
    }
# Usando a função de scoring no GridSearchCV
clf = GridSearchCV(GradientBoostingClassifier(), parameters,scoring=scoring,refit=False,cv=2, n_jobs=-1)
clf.fit(X_train, y_train)

#Convertendo clf.cv_results em um dataframe
df=pd.DataFrame.from_dict(clf.cv_results_)
#here Possible inputs for cross validation is cv=2, there two split split0 and split1
df[['split0_test_accuracy','split1_test_accuracy','split0_test_precision','split1_test_precision','split0_test_recall','split1_test_recall']]

# Encontrando o melhor parâmetro com base no accuracy_score
# Obtendo a média do accuracy_score
df['accuracy_score']=(df['split0_test_accuracy']+df['split1_test_accuracy'])/2
df.to_excel('criterios.xlsx', index=False)
parametros = df.loc[df['accuracy_score'].idxmax()]['params']

Wall time: 19min 23s


{'criterion': 'friedman_mse',
 'learning_rate': 0.2,
 'loss': 'deviance',
 'max_depth': 8,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 31,
 'n_estimators': 5,
 'subsample': 0.85}

In [48]:
#Aplicando na base teste
clf = GradientBoostingClassifier(**parametros).fit(X_train,y_train.ravel())
clf.score(X_test,y_test)

0.7841

### 5. Acessando o artigo do Jerome Friedman (Stochastic) e pensando no nome dado ao Stochastic GBM, qual é a maior diferença entre os dois algoritmos?

A cada iteração uma subamostra é extraída aleatoriamente sem reposição do conjunto completo de dados de treinamento. Essa subamostra é então usada em vez de uma amostra completa para ajustar os _base learner_.